Inception V3

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# loss and eval metrics
# https://www.kaggle.com/akensert/resnet50-keras-baseline-model
from keras import backend as K

def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    class_weights = np.array([2., 1., 1., 1., 1., 1.])
    eps = K.epsilon()
    y_pred = K.clip(y_pred, eps, 1.0-eps)
    out = -(         y_true  * K.log(      y_pred) * class_weights
            + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    return K.mean(out, axis=-1)

def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for the this competition
    """
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)

def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    class_weights = K.variable([2., 1., 1., 1., 1., 1.])
    eps = K.epsilon()
    y_pred = K.clip(y_pred, eps, 1.0-eps)
    loss = -(        y_true  * K.log(      y_pred)
            + (1.0 - y_true) * K.log(1.0 - y_pred))
    loss_samples = _normalized_weighted_average(loss, class_weights)
    return K.mean(loss_samples)

def weighted_log_loss_metric(trues, preds):
    """
    Will be used to calculate the log loss 
    of the validation set in PredictionCheckpoint()
    ------------------------------------------
    """
    class_weights = [2., 1., 1., 1., 1., 1.]
    epsilon = 1e-7
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_samples = np.average(loss, axis=1, weights=class_weights)
    return - loss_samples.mean()

Using TensorFlow backend.


In [3]:
train = pd.read_csv('train.csv', index_col=0)
val = pd.read_csv('val.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [4]:
print(train.shape, val.shape, test.shape)

(538630, 8) (68290, 8) (67337, 8)


In [5]:
538630 + 68290 + 67337 # 674257

674257

In [6]:
train.head() # they should be int not float
# need to add the .jpg to ID

,filename,PatientID,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_231d901c1.jpg,ID_b81a287f,1,0,0,0,1,0
2,ID_127689cce.jpg,ID_42910d3d,0,0,0,0,0,0
3,ID_25457734a.jpg,ID_329aafa7,0,0,0,0,0,0
4,ID_81c9aa125.jpg,ID_6b544c3c,0,0,0,0,0,0
5,ID_87e8b2528.jpg,ID_d6e578fb,0,0,0,0,0,0


In [7]:
# no test set - combine train and val
train = pd.concat([train, val], ignore_index=True)
train.shape

(606920, 8)

In [8]:
# need to create submission df
submission = pd.read_csv('stage_1_sample_submission.csv')
submission["Image"] = submission["ID"].str.slice(stop=12) + '.jpg'
submission["Diagnosis"] = submission["ID"].str.slice(start=13)
submission = submission.loc[:, ["Label", "Diagnosis", "Image"]]
submission = submission.set_index(['Image', 'Diagnosis']).unstack(level=-1)

In [9]:
submission.columns = submission.columns.droplevel(0)

In [10]:
submission = submission.reset_index()

In [11]:
submission.head()

Diagnosis,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000012eaf.jpg,0.5,0.5,0.5,0.5,0.5,0.5
1,ID_0000ca2f6.jpg,0.5,0.5,0.5,0.5,0.5,0.5
2,ID_000259ccf.jpg,0.5,0.5,0.5,0.5,0.5,0.5
3,ID_0002d438a.jpg,0.5,0.5,0.5,0.5,0.5,0.5
4,ID_00032d440.jpg,0.5,0.5,0.5,0.5,0.5,0.5


In [12]:
len(submission) # 78545

78545

In [13]:
batch_size = 32
im_size = 224
seed = 600
columns=["any", "epidural", "intraparenchymal", "intraventricular", "subarachnoid", "subdural"]

In [14]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range = 90,
    zoom_range = 0.1,
    horizontal_flip=True,
    vertical_flip=True
    )

In [15]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory='/home/jupyter/train_images_bsb_224/',
    x_col="filename",
    y_col=columns,
    target_size=(im_size, im_size),
    batch_size=batch_size,
    class_mode='other',
    seed=seed)

Found 606920 validated image filenames.


In [16]:
val_generator = train_datagen.flow_from_dataframe(
    dataframe=test,
    directory='/home/jupyter/train_images_bsb_224/',
    x_col="filename",
    y_col=columns,
    target_size=(im_size, im_size),
    batch_size=batch_size,
    class_mode='other',
    seed=seed)

Found 67337 validated image filenames.


In [17]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#test_generator = test_datagen.flow_from_dataframe(
 #   dataframe=test,
  #  directory='/home/jupyter/train_images_bsb_jpg/', # train/val/test are all in same dir
  #  x_col="filename",
  #  y_col=columns,
  #  target_size=(im_size, im_size),
  #  batch_size=batch_size,
  #  shuffle=False,
  #  class_mode='other',
  #  seed=seed)

In [18]:
submission_generator = test_datagen.flow_from_dataframe(
    dataframe=submission,
    directory='/home/jupyter/test_images_bsb_224/', # this one is different dir
    x_col="Image",
    y_col=None,
    target_size=(im_size, im_size),
    batch_size=batch_size,
    shuffle=False,
    class_mode=None,
    seed=seed
    )

Found 78545 validated image filenames.


In [22]:
num_train_steps = train_generator.n // train_generator.batch_size
num_val_steps = val_generator.n // val_generator.batch_size
num_epochs = 1

In [23]:
pre_trained_model = InceptionV3(input_shape=(224, 224, 3), 
                         weights='imagenet', 
                         include_top=False)

for layer in pre_trained_model.layers:
    layer.trainable = False

x = pre_trained_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)  
predictions = tf.keras.layers.Dense(6, activation='sigmoid')(x)
model = Model(inputs=pre_trained_model.input, outputs=predictions)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
# first use default lr to train top
model.compile(optimizer='rmsprop', loss=weighted_log_loss, metrics=[weighted_loss])

In [19]:
#need to use callback to save model
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='my_model.keras', save_best_only=True)

class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history = LossHistory()

In [20]:
# print to file
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')
#model.fit(X_train, Y_train, callbacks=[csv_logger])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_steps,
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=num_val_steps,
    callbacks=[checkpointer, csv_logger])

Epoch 1/2
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
13721/18966 [====================>.........] - ETA: 1:53:41 - loss: 0.2351 - weighted_loss: 0.2015

In [21]:
# load model
model = tf.keras.models.load_model('my_model.keras', custom_objects={'weighted_log_loss': weighted_log_loss, 'weighted_loss': weighted_loss})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
for layer in model.layers:
    layer.trainable = True

In [23]:
num_train_steps = train_generator.n // train_generator.batch_size
num_val_steps = val_generator.n // val_generator.batch_size
num_epochs = 40

In [24]:
# now recomplie use adam with small lr to train lower layers
model.compile(optimizer=Adam(lr=0.0001), loss=weighted_log_loss, metrics=[weighted_loss]) # change to 0.0005 if too slow to improve

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_steps,
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=num_val_steps,
    callbacks=[checkpointer, csv_logger])

Epoch 1/40
18966/18966 [==============================] - 6984s 368ms/step - loss: 0.1333 - weighted_loss: 0.1143 - val_loss: 0.1093 - val_weighted_loss: 0.0937
Epoch 2/40
18966/18966 [==============================] - 6873s 362ms/step - loss: 0.1059 - weighted_loss: 0.0908 - val_loss: 0.1025 - val_weighted_loss: 0.0879
Epoch 3/40
18966/18966 [==============================] - 6846s 361ms/step - loss: 0.0978 - weighted_loss: 0.0838 - val_loss: 0.0951 - val_weighted_loss: 0.0815
Epoch 4/40
18966/18966 [==============================] - 6810s 359ms/step - loss: 0.0932 - weighted_loss: 0.0799 - val_loss: 0.0938 - val_weighted_loss: 0.0804
Epoch 5/40
18966/18966 [==============================] - 6897s 364ms/step - loss: 0.0900 - weighted_loss: 0.0772 - val_loss: 0.0908 - val_weighted_loss: 0.0778
Epoch 6/40
18966/18966 [==============================] - 6959s 367ms/step - loss: 0.0876 - weighted_loss: 0.0751 - val_loss: 0.0924 - val_weighted_loss: 0.0792
Epoch 7/40
 4381/18966 [=====>....